# **Final Project**

This is a project doing some basic data analysis of IMDB movie data and associated wiki streaming events. It should be completed by groups of no less than 2 students and no more than 4 students. Each member of the group should have at least a few commits associated in the project repo.

## **Scoring**

The code must run and provide the correct answers . 1/2 points
The remainder will come from notebook organization, code comments, etc .
For the questions that have answers, please also provide those in markdown cells in the notebook, and/or part of a mardown file in the repo .
All relevant code should be shared via a shared Git repository. Additionally, you will send an email to joe@adaltas.com when the project has been submitted . Please ensure that the names of all participants are included in the repo and in the submission email . Note: For full credit the code must run with little to no extra input from the end user, and, any extra input that is required must be clearly documented and explained. Also note, any question that is at least attempted will be awarded with partial credit provided there is a corresponding explanation of the difficulties faced.

## **Questions**

  1 - load data from here. This should be done using a notebook cell and not a manual process to import the data. NOTE: You may not need all of the datasets, but you will be utilizing most of them.

  2 - How many total people in data set?

  3 - What is the earliest year of birth?

  4 - How many years ago was this person born?

  5 - Using only the data in the data set, determine if this date of birth correct.

  6 - Explain the reasoning for the answer in a code comment or new markdown cell.

  7 - What is the most recent data of birth?

  8 - What percentage of the people do not have a listed date of birth?

  9 - What is the length of the longest "short" after 1900?

  10 - What is the length of the shortest "movie" after 1900?

  11 - List of all of the genres represented.

  12 - What is the higest rated comedy "movie" in the dataset? Note, if there is a tie, the tie shall be broken by the movie with the most votes.

  13 - Who was the director of the movie?

  14 - List, if any, the alternate titles for the movie.

## **Stream Processing**

Choose any five entities from the data set. These can be specific movies, actors, crews, etc, or more abstract concepts such as specific genres, etc. The main criteria is that the entities chosen must have a trackable wiki page. Set up a stream processing job that will track events for the chosen entities from the wikimedia Events Platform. These tracking jobs should provide some simple metrics. These metrics should be stored in a database or file (depending on the platform used). At least one of the metrics should be of the "alert" type (meaning some event that would require further action. For instance imagine wanting to be notified each time a specific user makes a change. Capture this "alert" and mimic an alerting system by routing these events to a different file/database.) These tables/data do not need to be shared, but the structure of the output should be clearly noted in the code and/or markdown cells. Additionally, a brief explanation/overview of this section should be provided in a seperate markdown cell or in the project readme.

---

## **Population Script**

In [0]:
# 1. Create folder if not exists
import os

local_path = "/Workspace/Users/mael.depreville@edu.ece.fr/big-data-processing-project/data"

os.makedirs(local_path, exist_ok=True)
local_path

# 2. Download files using shell 
files = [
    "name.basics.tsv.gz",
    "title.akas.tsv.gz",
    "title.basics.tsv.gz",
    "title.crew.tsv.gz",
    "title.episode.tsv.gz",
    # "title.principals.tsv.gz",                        # Too big for instance to run it in Databricks
    "title.ratings.tsv.gz"
]

dict_files_names = {
    "name.basics.tsv.gz": "name.basics",
    "title.akas.tsv.gz": "title.akas",
    "title.basics.tsv.gz": "title.basics",
    "title.crew.tsv.gz": "title.crew",
    "title.episode.tsv.gz": "title.episode",
    # "title.principals.tsv.gz": "title.principals",    # Too big for instance to run it in Databricks
    "title.ratings.tsv.gz": "title.ratings"
}

base_url = "https://datasets.imdbws.com/"

for f in files:
    url = base_url + f
    out = f"{local_path}/{f}"
    print("Downloading:", f)
    os.system(f"wget -O {out} {url}")


In [0]:
dict_df = {}

for file_name in files:
    df = (spark.read
          .option("compression", "gzip")
          .option("inferSchema", "false")
          .option("nullValue", "\\N")
          .csv(f"{local_path}/{file_name}", header=True, sep="\t"))
    df.show(5)
    dict_df[dict_files_names[file_name]] = df

---

## **Questions**

**2.**

In [0]:
unique_name_count = (
  dict_df["name.basics"]
  .select("primaryName")
  .distinct()
  .count()
)
display(unique_name_count)

**3.**

There is a major problem here since the original dataset provides us with dates in absolute values as the example below proves it with the date of birth of Cesar (-100 -> 100).

In [0]:
raw = spark.read.text(f"{local_path}/name.basics.tsv.gz")
raw.filter(raw.value.contains("Gaio Giulio Cesare")).show(20, False)

-> We have then the date of birth closest to 0 in the code below.

In [0]:
from pyspark.sql.functions import col, min

min_birth_year = (
  dict_df["name.basics"]
  .select(min(col("birthYear").cast("double")))
  .collect()[0][0]
)

min_birth_year_df = dict_df["name.basics"].filter(col("birthYear") == min_birth_year)

min_birth_year_df.show(5)

**4.**

In [0]:
from datetime import date

current_year = date.today().year
years_difference = int(current_year - min_birth_year)
print(f"The difference between the current year and the earliest date of birth in our dataset is {years_difference} years!")

**5.**

**6.**

- If we consider this question as a question about the veracity of the earliest date of birth in this dataset, we have answered it a bit above (indicating that the date values were absolute -> preventing us from finding the earliest one but allowing us to find the closest to 0)

- On another hand, if we consider this question as a question about how we can check the veracity of the date of birth of this person

**7.**

In [0]:
from pyspark.sql.functions import col, max

max_birth_year = (
  dict_df["name.basics"]
  .select(max(col("birthYear").cast("double")))
  .collect()[0][0]
)

max_birth_year_df = dict_df["name.basics"].filter(col("birthYear") == max_birth_year)

max_birth_year_df.show(5)

**8.**

In [0]:
from pyspark.sql.functions import col

total_rows = dict_df["name.basics"].count()
null_rows = dict_df["name.basics"].filter(col("birthYear").isNull()).count()

birth_year_null_pct = (null_rows / total_rows) * 100

print(f"{birth_year_null_pct:.2f}% of the people in this dataset do not have a listed date of birth!")

**9.**

In [0]:
from pyspark.sql.functions import col, max

longest_short_after_1900 = (
  dict_df["title.basics"]
  .filter((col("titleType") == "short") & (col("startYear") >= 1900))
  .select(max(col("runtimeMinutes")))
  .collect()[0][0]
)

print(f"The longest short film after 1900 was {longest_short_after_1900} minutes long!")

**10.**

In [0]:
from pyspark.sql.functions import col, min

shortest_movie_after_1900 = (
  dict_df["title.basics"]
  .filter((col("titleType") == "movie") & (col("startYear") >= 1900))
  .select(min(col("runtimeMinutes")))
  .collect()[0][0]
)

print(f"The shortest movie film after 1900 was {shortest_movie_after_1900} minutes long!")

**11.**

In [0]:
from pyspark.sql.functions import split, explode, trim, col

genres_df = (
    dict_df["title.basics"]
    .select(explode(split(col("genres"), ",")).alias("genre"))
    .select(trim(col("genre")).alias("genre"))
    .filter(col("genre").isNotNull() & (col("genre") != ""))
)

unique_genres = [row["genre"] for row in genres_df.select("genre").distinct().collect()]

print(unique_genres)

**12.**

In [0]:
from pyspark.sql.functions import col, desc, dense_rank
from pyspark.sql.window import Window

df_joined = dict_df["title.basics"].join(
    dict_df["title.ratings"],
    on="tconst",
    how="inner"
)

df_filtered = df_joined.filter((col("titleType") == "movie") & (col("genres").contains("Comedy")))
w = Window.orderBy(desc("averageRating"), desc("numVotes"))

highest_rated_comedy_movie = (
    df_filtered
    .withColumn("rank", dense_rank().over(w))
    .filter(col("rank") == 1)
    .drop("rank")
)

highest_rated_comedy_movie.show(truncate=False)

**13.**

In [0]:
highest_rated_comedy_movie_director_df = (
    highest_rated_comedy_movie
    .join(dict_df["title.crew"], on="tconst", how="inner")
    .join(dict_df["name.basics"], dict_df["title.crew"].directors == dict_df["name.basics"].nconst, how="inner")
    .select("primaryName").alias("Director")
)

display(highest_rated_comedy_movie_director_df)

**14**

For this last question we suspect that it was based on the fields provided in the _title.principals_ dataset (that we couldn't download at the beginning of this notebook since it causes OOM in Databricks...)

But we could easily imagine that the query to obtain the alternate titles would be something like:  
<br>
```
highest_rated_comedy_movie_titles_df = (
    highest_rated_comedy_movie
    .join(dict_df["title.principals"], on="tconst", how="inner")
    .select("primaryTitle", "alternatesTitle")
)

display(highest_rated_comedy_movie_titles_df)
```

---

## **Stream Processing**